<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap12_generative_model_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q accelerate==0.31.0 peft==0.11.1 bitsandbytes==0.43.1 transformers==4.41.2 trl==0.9.4 sentencepiece==0.2.0 triton==3.1.0 datasets==2.20.0

12.3 QLoRA による指示チューニング

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset

# チャットテンプレートを使用するためのトークナイザーを読み込む
template_tokenizer = AutoTokenizer.from_pretrained(
    "TinyLlama/TinyLlma-1.1BChat-v1.0"
)

def format_prompt(example):
  """TinyLLmaが使用している<|user|>テンプレートを用いてプロンプトをフォーマットする"""

  # 回答をフォーマットする
  chat = example["messages"]
  prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)

  return {"text": prompt}

# データセットを読み込み、フォーマットする
dataset = (
    load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")
    .shuffle(seed=42)
    .select(range(3_000))
)
dataset = dataset.map(format_prompt)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: TinyLlama/TinyLlma-1.1BChat-v1.0 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:
# フォーマットされたテンプレートの例
print(dataset["text"][2576])

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "TinyLlma/TinyLlma-1.1B-intermediate-step-1431k-3T"

# 4ビット量子化設定(QLoRA の Q)
bnb_config = BitsAndBytesConfig(
    laod_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

# GPU で学習するためにモデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
 )
model.config.use_cashe = False
model.config.pretraining_tp = 1

# トークナイザーを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

#LoRA の設定を準備する
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj", "down_proj"]
)

# 学習用にモデルを準備する
model =  prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

12.3.4 学習の設定

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"

# 学習用の引数
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
    report_to="none"
)

In [ ]:
from trl import  SFTTrainer

# 教師ありファインチューニングのパラメーターを設定する
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=512,
    perf_cofig=peft_config,
)

trainer.train()

trainer.model.save_pretrained("TinyLlma-1.1B-qlora")

In [ ]:
12.3.6 重みのマージ

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto"
)

# LoRA とベースモデルをマージする
merged_model = model.merge_and_unload()

In [ ]:
from transformers import pipeline

# 事前定義されたプロンプトテンプレートを使用する
prompt = """<|user|>
大規模言語モデルについて教えてください。</s>
<|assistant|>
"""

# 指示チューニングされたモデルを実行する
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

In [ ]:
from datasets import load_dataset

def format_prompt(example):
  """Tiny Llma が使用している<|user|>テンプレートを使ってプロンプトをフォーマットする"""

  # 回答をフォーマットする
  system = "<|system|>\n" + example["system"] + "</s>\n"
  prompt = "<|user|>\n" + example["input"] + "</s>\n<|assistant|>\n"
  chosen = example["chosen"] + "</s>\n"
  rejected = example["rejected"] + "</s>\n"

  return {
      "prompt": system + prompt,
      "chosen": chosen,
      "rejected": rejected,
  }

# データセットにフォーマットを適用し、比較的短い回答を選択する
dpo_dataset = load_dataset(
    "agrilla/destilabel-intel-orca-dpo-pairs", split="train"
)
dpo_dataset = dpo_dataset.filter(
    lamdba r:
      r["status"] != "tie" and
      r["chosen_score"] >= 8 and
      not r["in_gsm8k_train"]
)
dpo_dataset = dpo_dataset.map(
    format_prompt,
    remove_columns=dpo_dataset.column_names
)
dpo_dataset

12.7.2 モデルの量子化

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import BitsAndBytesConfig, AutoTokenizer

# 4ビット量子化設定（QLoRAのQ）
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_double_quant=True,
)

# LoRA とベースモデルをマージ
model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto",
    quantization_config=bnb_config,
)
merged_model = model.merge_and_unload()

# LLaMA トークナイザーを読み込む
model_name = "TinyLlama/TinyLlma-1.1B-intermediate-step-1431k-3T"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

In [ ]:
from peft import LoraCofig, prepare_model_for_kbit_training, get_peft_model

# LoRA の設定を準備する
peft_config = LoRAConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj", "down_proj"]
)

# 学習用のモデルを準備する
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)